In [84]:
import pandas as pd
import re
import plotly.graph_objects as go
import datetime

# Funkcja do usunięcia dwukropka na końcu wartości
def remove_last_character(value):
    pattern = r':$'  # Wzorzec do dopasowania ostatniego dwukropka
    return re.sub(pattern, '', value)

# Funkcja do konwersji liczby sekund na format hh:mm:ss
def format_time(seconds):
    minutes, seconds = divmod(seconds, 60)
    hours, minutes = divmod(minutes, 60)
    return f'{int(hours)}:{int(minutes):02d}:{int(seconds):02d}'

# Wczytanie danych z pliku CSV
data = pd.read_csv('men_marathon.csv')

# Konwersja kolumny 'Event Date' na typ daty
data['Event Date'] = pd.to_datetime(data['Event Date'], format='%d.%m.%Y')

# Usunięcie dwukropka na końcu wartości w kolumnie 'Time'
data['Time'] = data['Time'].apply(remove_last_character)

# Konwersja kolumny 'Time' na timedelta
data['Time'] = pd.to_timedelta(data['Time'])

# Dodanie nowej kolumny 'Year' na podstawie kolumny 'Event Date'
data['Year'] = data['Event Date'].dt.year

# Filtrowanie danych od roku 1975
data = data[data['Year'] >= 1975]

# Grupowanie danych według roku i wybór najlepszego wyniku dla każdego roku
best_results = data.groupby('Year')['Time'].min().reset_index()

# Wyszukanie najlepszych zawodników dla każdego roku
best_athletes = data.merge(best_results, on=['Year', 'Time'])

# Utworzenie słownika przypisującego kolor dla każdego zawodnika
athlete_colors = {
    'Eliud Kipchoge': 'red',
    'Kelvin Kiptum': 'blue',
    'Kenenisa Bekele': 'green',
    'Dennis Kimetto': 'orange',
    'Titus Ekiru': 'purple',
    'Evans Chebet': 'yellow',
    'Geoffrey Mutai': 'pink',
    'Wilson Kipsang Kiprotich': 'gray',
    'Haile Gebrselassie': 'brown',
    'James Kipsang Kwambai': 'cyan',
    'Patrick Makau': 'magenta',
    'Paul Tergat': 'lime',
    'Khalid Khannouchi': 'olive',
    'Ronaldo da Costa': 'teal',
    'Felix Limo': 'navy',
    'António Pinto': 'maroon',
    'Belayneh Dinsamo': 'silver',
    'Josephat Kiprono': 'gold',
    'Sammy Lelei': 'indigo',
    'Cosmas N\'Deti': 'ivory',
    'Taisuke Kodama': 'violet',
    'Juma Ikangaa': 'coral',
    'Steve Jones': 'lavender',
    'David Tsebe': 'turquoise',
    'Steve Moneghetti': 'tan',
    'Robert de Castella': 'aqua',
    'Takeyuki Nakayama': 'salmon',
    'Martin Fiz': 'beige',
    'Alberto Salazar': 'khaki',
    'Dionisio Cerón': 'bisque',
    'Koichi Morishita': 'azure',
    'Gerard Nijboer': 'chartreuse',
    'Shigeru Soh': 'crimson',
    'Bill Rodgers': 'darkblue',
    'Carlos Lopes': 'darkgreen',
    'Waldemar Cierpinski': 'darkorange'
}

# Tworzenie wykresu za pomocą Plotly
fig = go.Figure()

# Dodawanie punktów na wykresie dla każdego zawodnika
for index, row in best_athletes.iterrows():
    athlete = row['Athlete']
    year = row['Year']
    time = row['Time']
    country = row['Country']
    time_str = format_time(time.total_seconds())
    label = f"Zawodnik: {athlete}<br>Rok: {int(year)}<br>Czas: {time_str}<br>Kraj: {country}"
    color = athlete_colors.get(athlete, 'blue')  # Pobieranie koloru dla zawodnika, domyślnie 'blue'
    fig.add_trace(go.Scatter(
        x=[year],
        y=[time.total_seconds()],
        mode='markers',
        name=athlete,
        text=[label],
        hovertemplate='%{text}<extra></extra>',
        marker=dict(size=15, color=color),
    ))

# Konfiguracja układu wykresu
fig.update_layout(
    xaxis=dict(title='Rok'),
    yaxis=dict(
        title='Najlepszy czas',
        tickvals=[7800, 7680, 7560, 7440, 7320],  # Wartości osi Y
        ticktext=['2:10:00', '2:08:00', '2:06:00', '2:04:00', '2:02:00'],  # Etykiety osi Y
    ),
    title='Najlepszy wynik w biegu na 100 m',
    showlegend=True,
)

# Zapisywanie wykresu do pliku HTML
fig.write_html('best_of_every_season.html')



In [89]:
import csv
from datetime import datetime
import re
import pandas as pd
import numpy as np
import plotly.graph_objects as go

def calculate_age(birth_date, event_date):
    birth_date = datetime.strptime(birth_date, "%d.%m.%y")
    event_date = datetime.strptime(event_date, "%d.%m.%Y")
    
    birth_year = birth_date.year
    event_year = event_date.year
    
    if birth_year > 2010:
        birth_year = birth_year - 100
    
    age = event_year - birth_year
    if event_date.month < birth_date.month or (event_date.month == birth_date.month and event_date.day < birth_date.day):
        age -= 1
    return age


def remove_last_character(value):
    pattern = r'\D$'  # Wzorzec do dopasowania ostatniego niecyfrowego znaku
    return re.sub(pattern, '', value)


# Wczytanie danych z pliku CSV
data = pd.read_csv('men_marathon.csv')

# Usunięcie litery na końcu wartości w kolumnie 'Time'
data['Time'] = data['Time'].apply(remove_last_character)

# Konwersja kolumny 'Time' na liczbę zmiennoprzecinkową
data['Time'] = data['Time'].str.split(':').apply(lambda x: int(x[0]) * 3600 + int(x[1]) * 60 + int(x[2]))

# Znalezienie najlepszego czasu dla każdego zawodnika
best_times = data.groupby('Athlete')['Time'].min()

all_ages = []
# Iteracja po zawodnikach i ich najlepszych czasach
for athlete, best_time in best_times.items():
    athlete_data = data[(data['Athlete'] == athlete) & (data['Time'] == best_time)]
    
    birth_date = athlete_data['Date of Birth'].values[0]
    event_date = athlete_data['Event Date'].values[0]

    # Sprawdzenie, czy birth_date lub event_date jest puste lub nie jest prawidłową datą
    if pd.isnull(birth_date) or pd.isnull(event_date) or not isinstance(birth_date, str) or not isinstance(event_date, str):
        continue
    
    try:
        age = calculate_age(birth_date, event_date)
        all_ages.append(age)
    except ValueError:
        continue

unique_values, frequencies = np.unique(all_ages, return_counts=True)
value_to_delete = 83

# Znalezienie indeksu wartości do usunięcia
index_to_delete = np.where(unique_values == value_to_delete)[0]
unique_values = np.delete(unique_values, index_to_delete)
frequencies = np.delete(frequencies, index_to_delete)

# Tworzenie wykresu w Plotly
fig = go.Figure()

# Dodawanie słupków na wykres
fig.add_trace(go.Bar(
    x=unique_values,
    y=frequencies,
    marker_color='#4287f5',  # Kolor słupków
    text=frequencies,
    textposition='outside',  # Wyświetlanie tekstu nad słupkiem
    textfont=dict(size=12, color='black')  # Konfiguracja czcionki tekstu
))

# Konfiguracja osi i tytułu wykresu
fig.update_layout(
    xaxis_title='Wiek biegacza',
    yaxis_title='Liczba biegaczy, którzy ustawili rekord osobisty',
    title='Wiek vs Personal Best',
    xaxis=dict(type='linear'),  # Wymuszenie liniowego układu osi X
    yaxis=dict(showgrid=True, gridcolor='lightgray'),  # Wyświetlanie siatki na osi Y
)

# Zapisywanie wykresu w pliku HTML
fig.write_html('personal_best_graph.html')


In [114]:
import pandas as pd
import plotly.graph_objects as go

# Funkcja do usunięcia litery na końcu wartości
def remove_last_character(value):
    pattern = r'\D$'  # Wzorzec do dopasowania ostatniego niecyfrowego znaku
    return re.sub(pattern, '', value)

# Funkcja do formatowania czasu
def format_time(seconds):
    minutes, seconds = divmod(seconds, 60)
    hours, minutes = divmod(minutes, 60)
    return f'{int(hours)}:{int(minutes):02d}:{int(seconds):02d}'

# Wczytanie danych z pliku CSV
data = pd.read_csv('men_marathon.csv')

# Konwersja kolumny 'Event Date' na typ daty
data['Event Date'] = pd.to_datetime(data['Event Date'], format='%d.%m.%Y')

# Usunięcie litery na końcu wartości w kolumnie 'Time'
data['Time'] = data['Time'].apply(remove_last_character)

# Konwersja kolumny 'Time' na liczbę zmiennoprzecinkową
data['Time'] = data['Time'].str.split(':').apply(lambda x: int(x[0]) * 3600 + int(x[1]) * 60 + int(x[2]))

# Sortowanie wyników względem daty i czasu
data = data.sort_values(by=['Event Date', 'Time'])

# Inicjalizacja pustej listy do przechowywania najlepszych wyników
best_results = []

# Ustalenie daty początkowej
start_date = pd.to_datetime('1968-10-14', format='%Y-%m-%d')

# Przeszukiwanie posortowanych danych w celu znalezienia progresji rekordu świata
current_best_time = float('inf')
for _, row in data.iterrows():
    if row['Event Date'] >= start_date:
        if row['Time'] < current_best_time:
            best_results.append(row)
            current_best_time = row['Time']

# Konwersja listy wyników do DataFrame
best_results = pd.DataFrame(best_results)

# Tworzenie wykresu linii
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=best_results['Event Date'],
    y=best_results['Time'],
    mode='lines',
    name='Progresja rekordu świata - Maraton mężczyzn'
))

# Ustawienie opisów osi
fig.update_layout(
    xaxis=dict(title='Data wydarzenia'),
    yaxis=dict(title='Czas (hh:mm:ss)')
)

# Ustawienie osi Y
fig.update_yaxes(
    tickvals=[7800, 7680, 7560, 7440, 7320],
    ticktext=['2:10:00', '2:08:00', '2:06:00', '2:04:00', '2:02:00'],
    tickformatstops=[
        dict(dtickrange=[None, 1000], value="%H:%M:%S"),  # Formatowanie na hh:mm:ss dla małych przedziałów
        dict(dtickrange=[1000, None], value="%H:%M")  # Formatowanie na hh:mm dla większych przedziałów
    ]
)

# Dodanie tytułu wykresu
fig.update_layout(
    title="Progresja rekordu świata - Maraton mężczyzn"
)

# Zapisywanie wykresu do pliku HTML
fig.write_html("WR_progression.html")


